In [1]:
import tensorflow as tf
from utils.pretrained_glove import GloveTrainer
from utils.preprocessing import clean_tweets
import numpy as np
from model.dnc_trainer import DNCTrainer
from utils.argument_loader import ArgumentLoader
import dnc.dnc as dnc

/home/mezo/anaconda3/envs/ada/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
al = ArgumentLoader()
FLAGS = al.get_configuration()

In [3]:
trainable_variables = tf.trainable_variables()
trainable_variables

[]

In [4]:
init_op = tf.global_variables_initializer()

In [5]:
y_= tf.placeholder(tf.float32,shape=[100,20,2])
x = tf.placeholder(tf.float32, [100, 20,200])
mask = tf.placeholder(tf.float32,shape=[100,20])
#init_op = tf.initialize_all_variables()
trainable_variables = tf.trainable_variables()

sess = tf.Session()
#init_op = tf.global_variables_initializer()
saver = tf.train.import_meta_graph('./checkpoints/tmp1/model.ckpt-36000.meta')
# sess.run(init_op)
saver.restore(sess,tf.train.latest_checkpoint('./checkpoints/tmp1/'))
#sess.run(trainable_variables)
    #print(x.eval())
#First let's load meta graph and restore weights

INFO:tensorflow:Restoring parameters from ./checkpoints/tmp1/model.ckpt-36000


In [6]:
sess.run(init_op)

In [7]:
gt = GloveTrainer(vector_size="200", glove_dir="./glove.twitter.27B/")
word_embeddings = gt.generate_word_embeddings()

Indexing word vectors.


In [8]:
loadedInstances = 0
test_tweets = open("./twitter-datasets/test_data.txt","r")
t_tweets = []
while loadedInstances < 10000:
    raw_test_tweets = test_tweets.readline()
    raw_test_tweets = clean_tweets(raw_test_tweets)
    t_tweets.append(raw_test_tweets)
    loadedInstances = loadedInstances + 1 
test_tweets.close()

In [9]:
missing_voc = {}
tweet_glove = [0]*10000
for i in range(len(t_tweets)):
    matrix_embedding = []
    for word in t_tweets[i].split():
        try:
            matrix_embedding.append(word_embeddings[word])
        except:
            try:
                missing_voc[word] = missing_voc[word] + 1
            except KeyError:
                missing_voc[word] = 1
                    
    tweet_glove[i] = ((matrix_embedding),0)

In [10]:
def get_generator(training_set):
    word_size = 200
    batch_size = 100
    max_lenght = 20
    num_classes = 2

    batched_input = []
    batched_label = []
    batched_mask = []

    i = 0
    num_batch = 0
    batch_completed = False
    while i < (len(training_set) + 1):
        if not batch_completed:
            if len(batched_input) < batch_size:
                data = training_set[i][0]
                data = data[:max_lenght]
                if len(data) > 0:
                    padded = np.lib.pad(data, ((max_lenght - len(data), 0), (0, 0)), 'constant', constant_values=(0))
                    delta = 1.0 / (len(data))
                else:
                    padded = np.zeros(shape=[max_lenght, word_size])
                    delta = 1.0
                batched_input.append(padded)
                if True:
                    label = np.zeros(shape=[max_lenght,num_classes])
                    for j in range(0,max_lenght):
                        np.put(label[j],training_set[i][1],1.0)
                    batched_label.append(label)
                    mask = np.zeros(max_lenght)
                    weight = np.zeros(len(data))
                    for number in range(0, len(data)):
                        np.put(weight, number, delta * (number + 1))
                    np.put(mask, np.arange(max_lenght - len(data), max_lenght), weight)
                    batched_mask.append(mask)
                    i += 1
            else:
                reshaped = np.reshape(batched_input, newshape=[batch_size, max_lenght, word_size])
                labels = np.reshape(batched_label, newshape=[batch_size, max_lenght, num_classes])
                masks = np.reshape(batched_mask, newshape=[batch_size, max_lenght])
                toAppend = (reshaped, labels, masks)
                num_batch += 1
                if (len(training_set) // batch_size) == num_batch:
                    batch_completed = True
                batched_input = []
                batched_label = []
                batched_mask = []
                yield toAppend
        else:
            break

In [ ]:
sess.run(tf.variables_initializer(set(tf.all_variables())))

In [16]:
final_prediction_asd = []
for iteration in range(0, 100):
    dnc_trainer = DNCTrainer(FLAGS)
    final_tweets = next(get_generator(tweet_glove))
    temp = set(tf.global_variables())
    output_logits=(dnc_trainer.run_model(final_tweets[0]))
    sess.run(tf.variables_initializer(set(tf.all_variables()) - temp))
    prediction = tf.argmax(output_logits[:,FLAGS.max_lenght-1], 1)
    #predicted_batch = sess.run(prediction)
    final_prediction_asd.append(prediction)
    print(len(final_tweets[0]))
    print(str(iteration) + " done!")
    #if iteration > 1:
            #break

Running model
Instructions for updating:
Please use tf.global_variables instead.
100
0 done!
Running model
100
1 done!
Running model
100
2 done!
Running model
100
3 done!
Running model
100
4 done!
Running model
100
5 done!
Running model
100
6 done!
Running model
100
7 done!
Running model
100
8 done!
Running model
100
9 done!
Running model
100
10 done!
Running model
100
11 done!
Running model
100
12 done!
Running model
100
13 done!
Running model
100
14 done!
Running model
100
15 done!
Running model
100
16 done!
Running model
100
17 done!
Running model
100
18 done!
Running model
100
19 done!
Running model
100
20 done!
Running model
100
21 done!
Running model
100
22 done!
Running model
100
23 done!
Running model
100
24 done!
Running model
100
25 done!
Running model
100
26 done!
Running model
100
27 done!
Running model
100
28 done!
Running model
100
29 done!
Running model
100
30 done!
Running model
100
31 done!
Running model
100
32 done!
Running model
100
33 done!
Running model
100
34 done

KeyboardInterrupt: 

In [ ]:
sess.run(final_prediction_asd[0])

In [ ]:
sess.run(final_prediction_asd[0])

In [11]:
trainable_variables = tf.trainable_variables()

In [12]:
trainable_variables

[<tf.Variable 'dnc/lstm/w_gates:0' shape=(656, 1024) dtype=float32_ref>,
 <tf.Variable 'dnc/lstm/b_gates:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/write_vectors/w:0' shape=(256, 200) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/write_vectors/b:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/erase_vectors/w:0' shape=(256, 200) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/erase_vectors/b:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/free_gate/w:0' shape=(256, 1) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/free_gate/b:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/allocation_gate/w:0' shape=(256, 1) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/allocation_gate/b:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/write_gate/w:0' shape=(256, 1) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/write_gate/b:0' shape=(1,) dtype=float32_ref>,
 <tf.Varia

In [13]:
a = tf.global_variables( )

In [14]:
a

[<tf.Variable 'dnc/lstm/w_gates:0' shape=(656, 1024) dtype=float32_ref>,
 <tf.Variable 'dnc/lstm/b_gates:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/write_vectors/w:0' shape=(256, 200) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/write_vectors/b:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/erase_vectors/w:0' shape=(256, 200) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/erase_vectors/b:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/free_gate/w:0' shape=(256, 1) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/free_gate/b:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/allocation_gate/w:0' shape=(256, 1) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/allocation_gate/b:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/write_gate/w:0' shape=(256, 1) dtype=float32_ref>,
 <tf.Variable 'dnc/memory_access/write_gate/b:0' shape=(1,) dtype=float32_ref>,
 <tf.Varia

In [15]:
len(a)

73

In [ ]:
print(sess.run(tf.report_uninitialized_variables()))

In [ ]:
list_names = sess.run(tf.report_uninitialized_variables())

In [ ]:
len(list_names)

In [ ]:
tf.get_variable(list_names[0])

In [ ]:
trainable_variables

In [ ]:
sess.run(final_prediction_asd[0])

In [ ]:
final = []
for i in range(0, 100):
    final.extend(sess.run(final_prediction_asd[i]))


In [ ]:
for i,n in enumerate(final):
    if n == 0:
        final[i]=-1

In [ ]:
ids = np.arange(1,10001)

In [ ]:
ids

In [ ]:
import csv
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})

In [ ]:
create_csv_submission(ids,final,'First_submission.csv')